# PROJET : 
# RECONNAISSANCE VOCALE
# SYSTEME DE TRADUCTION ADAPTE AUX LUNETTES CONNECTEES  
  
    
      
***

   
      
   
## Partie II : Modélisation - Itération 1  
   
   


Dans cette itération, nous avons réalisé notre première modélisation, faisant suite à l'étape de visualisation. 

L'hypothèse de base de cette itération est l'hypothèse suivante : 
"H0: dans une phrase donnée, les mots à traduire en anglais et en français sont quasiment à la même place. En s'appuyant sur cette logique, une traduction mot à mot d'un ensemble de phrases est possible."

Cette hypothèse naive, bien qu'intéressante, ne nous semble pas vraiment plausible.
En effet, dans l'étape de visualisation, nous avons constaté que le nombre de mots et la structure des phrases sont très différents en français et en anglais.

Nous allons toutefois essayer de trouver des solutions pour faire fonctionner au mieux une telle traduction et comparer les résultats obtenus avec le dictionnaire de référence fourni. 

In [1]:
# Import des données
import numpy as np
import pandas as pd

data = pd.read_csv("small_vocab_fr-eng.csv", sep = ';')

# Suppression des doublons 
data.drop_duplicates(inplace = True)
data.reset_index(inplace = True, drop = True)

data.head()

,Français,English
0,new jersey est parfois calme pendant l' automn...,"new jersey is sometimes quiet during autumn , ..."
1,les états-unis est généralement froid en juill...,the united states is usually chilly during jul...
2,"california est généralement calme en mars , et...","california is usually quiet during march , and..."
3,"les états-unis est parfois légère en juin , et...",the united states is sometimes mild during jun...
4,"votre moins aimé fruit est le raisin , mais mo...","your least liked fruit is the grape , but my l..."


   
### 1. Création d'un dictionnaire anglais - français   
   
   

La première étape de modélisation consiste à réaliser un dictionnaire de correspondance entre un mot anglais et un mot français. 
Nous avons choisi ce sens de réalisation du dictionnaire car c'est celui qui nous sera utile pour notre projet de lunettes connectées 
(la transcription à l'écrit des phrases orales étant réalisée en langue anglaise). 

Pour réaliser ce dictionnaire, nous avons procédé ainsi: 
- création de la liste des mots en anglais dont nous cherchons la traduction; 
- pour chaque mot anglais, en fonction de sa place dans la phrase, création d'une fenêtre de possibilité de trois mots (autour de la place du mot anglais) dans la version française;
- formalisation d'un tableau (array) de quatre colonnes fournissant, pour chaque mot anglais, les trois mots possibles en français;
- tri du tableau par mot anglais afin d'affecter à chaque mot anglais unique toutes les possibilités de traduction en francais;
- calcul de l'occurence d'apparition de chaque possibilité ;
- selection de la plus forte occurence pour chaque mot unique ce qui nous permet d'établir la correspondance souhaitée entre un mot anglais et un mot français. 

In [2]:
# A partir du dataframe, on crée une liste de phrases (par ligne de tableau) en anglais et en français

# Création de la liste de phrases en anglais
liste_eng = []
for i in range(len(data.index)):
    text_eng = ""
    for carac in data.English[i]:
        if carac in ",.?":
            text_eng += " "
        elif carac in "'":
            text_eng += " '"
        else:
            text_eng += carac
    liste_eng.append(text_eng)


# Création de la liste de phrases en français
liste_fr = []
for i in range(len(data.index)):
    text_fr = ""
    for carac in data.Français[i]:
        if carac in ",.?-":
            text_fr += " "
        elif carac in "'":
            text_fr += "' "
        else:
            text_fr += carac
    liste_fr.append(text_fr)



In [3]:
# Création de la liste de mots en anglais
liste_eng1 = []
for i in range(len(liste_eng)):
    for j in range(len(liste_eng[i].split())):
        liste_eng1.append(liste_eng[i].split()[j])


# Correction des éléments en forme contractée
for index, value in enumerate(liste_eng1):
    if value == "didn":
        liste_eng1[index] = 'did'
    elif value == "isn":
        liste_eng1[index] = 'is'
    elif value == "aren":
        liste_eng1[index] = 'are'
    elif value == "'s":
        liste_eng1[index] = 'is'
    elif value == "'t":
        liste_eng1[index] = 'not'

#liste_eng1

In [4]:
# Création de la liste des 3 mots en français en fonction de la position du mot anglais à traduire : 
liste_fr1 = []
for i in range(len(liste_eng)):
    k = len(liste_fr[i].split()) - 1
    l = len(liste_eng[i].split()) - 1
    
    # Création de la liste des mots par phrase anglaise
    for j in range(len(liste_eng[i].split())):
        
        # Liste de mots pour le premier mot de la phrase anglaise
        if j == 0:
            liste_fr1.append([liste_fr[i].split()[0], liste_fr[i].split()[1], liste_fr[i].split()[2]])
            
        # Liste des mots pour le dernier mot de la phrase anglaise   
        elif j == l: 
            liste_fr1.append([liste_fr[i].split()[-3], liste_fr[i].split()[-2], liste_fr[i].split()[-1]])
            
        # Liste des mots pour les mots centraux de la phrase anglaise
        else:
            
            # Phrases anglaises et françaises avec le même nombre de mots
            if k == l:
                liste_fr1.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                
                
            # Phrases anglaises avec plus de mots que les françaises
            elif k < l:
                if 1 <= j < k:
                    liste_fr1.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                else:
                    liste_fr1.append([liste_fr[i].split()[-3], liste_fr[i].split()[-2], liste_fr[i].split()[-1]])
                
                
            # Phrases françaises avec plus de mots que phrases anglaises
            else:
                liste_fr1.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])


#liste_fr1

In [5]:
# Vérification de la taille des listes obtenues
print("Taille de la liste anglaise liste_eng1 :",len(liste_eng1))
print("Taille de la liste française liste_fr1 :",len(liste_fr1))

Taille de la liste anglaise liste_eng1 : 1501574
Taille de la liste française liste_fr1 : 1501574


In [6]:
# Création d'un array reprenant les mots anglais et les trios de mots français
arr_eng = np.array(liste_eng1).reshape(1501574,1)
arr_fr = np.array(liste_fr1)
arr = np.hstack((arr_eng, arr_fr))
arr.shape

(1501574, 4)

In [7]:
# Réorganisation de l'array en fonction des mots anglais
arr1 = arr[arr[:, 0].argsort()]
arr1.shape

(1501574, 4)

In [8]:
# Création d'un array des mots anglais uniques avec leurs occurences
uniq_eng = np.asarray(list(zip(*np.unique(arr1[:,0], return_counts= True))))
print("Nombre de mots anglais uniques :", uniq_eng.shape[0])

Nombre de mots anglais uniques : 196


In [9]:
# Création du dictionnaire de traduction

# Dictionnaire de traduction anglais - français 
dico_trad = {} 

# Dictionnaire des possibles traductions suivant l'occurrence
dico_occ_nbr = {}

# Dictionnaire des possibles traductions suivant l'occurrence en pourcentage
dico_occ_pctg = {}

k1 = 0 
for i in range(uniq_eng.shape[0]):
    k2 = uniq_eng[i, 1].astype('int')
    arr2 = arr1[k1 : k1 + k2, 1:4]
    values, counts = np.unique(arr2, return_counts = True)
    dico_trad.update({uniq_eng[i, 0] : values[np.argmax(counts)]})
    dico_occ_nbr.update({uniq_eng[i, 0] : dict(zip(*np.unique(arr2, return_counts= True)))})
    dico_count = dict(zip(*np.unique(arr2, return_counts= True)))
    for key, value in dico_count.items():
        dico_count[key] = round(value / (k2 * 3) * 100, 2)
        dico_occ_pctg.update({uniq_eng[i, 0] : dico_count})
    k1 += k2

# Création du dictionnaire de traduction avec les différentes possibilités de traductions
dico_occ = {}
for i in dico_trad.keys():
        dico_occ.update({i : list(dico_occ_nbr[i].keys())})

# Création du dictionnaire de traduction en précisant le pourcentage d'occurrence du mot choisi
dico_trad_pctg = {}
for i in dico_trad.keys():
    for j in dico_occ_pctg[i].keys():
        if j == dico_trad[i]:
            dico_trad_pctg.update({i : [dico_trad[i], dico_occ_pctg[i][j]]})

dico_trad_pctg

{'a': ['une', 16.72],
 'am': ['je', 33.33],
 'and': ['et', 18.41],
 'animal': ['préféré', 16.67],
 'animals': ['les', 23.13],
 'apple': ['la', 33.33],
 'apples': ['les', 34.31],
 'april': ['en', 33.25],
 'are': ['sont', 23.95],
 'august': ['août', 17.14],
 'automobile': ['voiture', 26.85],
 'autumn': ['automne', 26.55],
 'banana': ['la', 33.33],
 'bananas': ['les', 37.14],
 'bear': ['ours', 33.33],
 'bears': ['les', 32.29],
 'beautiful': ['est', 19.39],
 'been': ['été', 31.48],
 'between': ['entre', 24.07],
 'big': ['grande', 13.48],
 'bird': ['oiseau', 33.33],
 'birds': ['oiseaux', 31.25],
 'black': ['voiture', 19.07],
 'blue': ['voiture', 17.11],
 'busy': ['occupé', 24.42],
 'but': ['mais', 26.55],
 'california': ['est', 31.37],
 'car': ['voiture', 33.33],
 'cat': ['chat', 33.33],
 'cats': ['chats', 32.81],
 'chilly': ['froid', 25.8],
 'china': ['chine', 33.05],
 'chinese': ['chinois', 18.59],
 'cold': ['froid', 26.51],
 'december': ['en', 33.2],
 'did': ['pourquoi', 15.15],
 'diffic

Dans l'étape de visualisation, nous avions calculé la présence de 196 mots unique en anglais, nous retrouvons ce nombre dans notre dictionnaire, ce qui nous assure une certaine cohérence. 

En parcourant visuellement la liste des 196 mots traduits, nous constatons plusieurs points : 
- les occurences d'apparition ne sont pas très élevées (jamais au-dessus de 50%);
- certaines traductions sont manifestement erronées. 

En première conclusion, et sans même comparer avec le dictionnaire de référence, nous pouvons présager de l'invalidité de l'hypothèse de base. 

Dans la suite de l'étude, nous appelerons ce doctionnaire "le 1er dictionnaire". 

Nous allons toutefois procéder à la comparaison et tâcher de trouver des pistes d'amélioration.

   
### 2. Etude du dictionnaire créé :  comparaison avec un dictionnaire de référence
   
   

Dans cette étape, nous construisons le dictionnaire de référence.

Mais tout d'abord, nous souhaiterions commenter les données d'entrée. 

Le dictionnaire proposé comme référence est très riche (113 286 mots). 
Toutefois, nous constatons des erreurs plus ou moins importantes: 
- Le premier biais est de parfois traduire un mot par un terme du même champs lexical (ex: cat par félin);
- Le deuxième de traduire un gérondif par un nom (ex: canoying par canoe);
- De nombreux mots sont incompréhensibles (ex: livi-livi ou heiliger-heiliger), ces mots sont généralement identiquement traduits en français et en anglais, ce qui ne causera pas de problèmes pour notre comparaison mais fait douter de la valeur de ce dictionnaire; 
- De façon moins anecdoctique, certains mots sont traduits une fois par le mot en français (ex: bird par oiseau) et une autre fois par le mot en anglais (ex: bird par bird), ou bien une fois par un nom et une fois par un adjectif (religion - religion et religion - religieux), ce qui, par contre, pourrait être problématique pour notre comparaison. 

Dans le code ci-après, nous construisons non pas un dictionnaire un pour un, c'est-à-dire ne proposant qu'une traduction pour chaque mot, mais un dictionnaire un pour multiples (via l'utilisation d'une liste), ce qui permet d'obtenir plusieurs choix de traduction pour un même mot. Cette méthode nous permet de nous affranchir des biais des données d'entrée.  

In [10]:
# Création du dictionnaire de référence à partir du 2ème dataset

    # Import des données du dataset 2
ds2 = pd.read_csv("en-fr.txt", sep = " ", names = ["Anglais", "Français"])
ds2_ord = ds2.groupby(['Anglais']).agg(lambda x : list(x)).reset_index()

    # Création du dictionnaire complet
dico_cplt = {ds2_ord['Anglais'][i] : ds2_ord['Français'][i] for i in range(ds2_ord.shape[0])}

    # Nombre de mots anglais différents dans ce dataset
print("Ce dataset constituant le dictionnaire de référence contient les traductions françaises de", len(dico_cplt),
      "mots anglais différents.")

    # Création du dictionnaire en reprenant les mots anglais en commun dans les 2 datasets
dico_part = {}
for i in dico_trad.keys():
    if i in dico_cplt.keys():
        dico_part.update({i : dico_cplt[i]})

print("Le dictionnaire créé dans la partie précédente et le dictionnaire de référence ont", len(dico_part),
      "mots en commun.")
#dico_part

liste_nc = []
for i in dico_trad.keys():
    if i not in dico_part:
        liste_nc.append(i)
print("Liste des mots qui n'apparaissent pas dans le dictionnaire de référence contenant", len(liste_nc), "mots:\n",
      liste_nc)

Ce dataset constituant le dictionnaire de référence contient les traductions françaises de 94679 mots anglais différents.
Le dictionnaire créé dans la partie précédente et le dictionnaire de référence ont 174 mots en commun.
Liste des mots qui n'apparaissent pas dans le dictionnaire de référence contenant 22 mots:
 ['a', 'am', 'been', 'chilly', 'disliked', 'dislikes', 'do', 'does', 'drives', 'go', 'has', 'he', 'i', 'in', 'is', 'it', 'least', 'my', 'snowy', 'thinks', 'to', 'we']


#### Analyse de la comparaison entre le dictionnaire dico_trad obtenu et le dictionnaire de référence 

In [11]:
# Comparaison de la traduction proposée dans le 1er dictionnaire et des traductions du dictionnaire de référence
dico_com1 = {}
liste_nt = []
for i in dico_part.keys():
    if dico_trad[i] in dico_part[i]:
        dico_com1.update({i : [dico_trad[i], dico_trad_pctg[i][1]]})
    else:
        liste_nt.append(i)
        
# Analyse du pourcentage d'occurrence des mots choisis pour la traduction
liste_pctg = [dico_com1[i][1] for i in dico_com1.keys()]
print("Les mots français les plus fréquents choisis pour la traduction des mots anglais du 1er dictionnaire ont une \
occurrence comprise entre", min(liste_pctg), "% et", max(liste_pctg), "%.")

# Analyse du score du 1er dictionnaire
print("Les traductions proposées dans le 1er dictionnaire correspondent à celles proposées par le dictionnaire de \
référence pour", len(dico_com1), "mots, soit dans", round(len(dico_com1)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots mal traduits, de", len(liste_nt), "mots :\n", liste_nt)


Les mots français les plus fréquents choisis pour la traduction des mots anglais du 1er dictionnaire ont une occurrence comprise entre 13.48 % et 33.33 %.
Les traductions proposées dans le 1er dictionnaire correspondent à celles proposées par le dictionnaire de référence pour 82 mots, soit dans 47.13 % des cas.

Liste des mots mal traduits, de 92 mots :
 ['animal', 'animals', 'apple', 'apples', 'april', 'automobile', 'banana', 'bananas', 'bears', 'beautiful', 'black', 'blue', 'california', 'december', 'did', 'dislike', 'driving', 'drove', 'during', 'eiffel', 'english', 'fall', 'feared', 'february', 'field', 'football', 'freezing', 'fruit', 'going', 'grape', 'grapefruit', 'grapes', 'green', 'grocery', 'have', 'her', 'january', 'july', 'june', 'last', 'lemon', 'lemons', 'like', 'liked', 'lime', 'limes', 'loved', 'mango', 'mangoes', 'march', 'may', 'might', 'most', 'new', 'next', 'nice', 'november', 'october', 'orange', 'oranges', 'peach', 'peaches', 'pear', 'pears', 'plan', 'plans', 'rus

De ces valeurs, nous établissons les constats suivants: 
- la plupart des mots du 1er dictionnaire sont bien présents dans le dictionnaire de référence mais certains en sont absents (22 mots);
- notre score de performance est assez faible (moins 50%), ce qui confirme que nous allons sûrement rejeter l'hypothèse H0;

Par la suite, nous allons pousser plus loin l'étude afin de trouver tout de même des pistes d'amélioration de ce score. 

In [12]:
# Comparaison entre les différentes possibilités de traductions du 1er dictionnaire et celles du dictionnaire de référence
dico_lt = {}
for i in dico_part.keys():
    for j in range(len(dico_occ[i])):
        if dico_occ[i][j] in dico_part[i]:
            dico_lt.update({i : dico_occ[i]})
            break
            
liste_lnt = []
for i in dico_part.keys():
    if i not in dico_lt.keys():
        liste_lnt.append(i)

print("Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt), "mots, soit dans", 
      round(len(dico_lt)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt), "mots :\n", liste_lnt)

Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 160 mots, soit dans 91.95 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 14 mots :
 ['did', 'dislike', 'drove', 'feared', 'field', 'have', 'her', 'most', 'nice', 'plan', 'plans', 'saw', 'store', 'would']


De ces analyses, nous extrayons les observations suivantes: 

- dans la grande majorité des cas, la bonne traduction de chaque mot anglais est bien dans la fenêtre de 3 mots sélectionnée dans la version française; 
- seuls 14 mots se trouvent en-dehors de cette fenêtre de 3 mots. 

Nous n'avons pas, dans ce cas, jugé utile de modifier la fenêtre de traduction, considérant cette piste d'amélioration de nos performances comme négligeable. En effet, ces mots sont mal traduits pour d'autres raisons (conjuguaison de verbes, autres façons de construire une phrase en anglais et en francais). 

#### Analyse d'un second dictionnaire créé sans certains mots français trop communs

Dans la poursuite de l'objectif d'amélioration des performances, nous avons testé le modèle en retirant des mots usuels listés dans "ban" et retrouvés trop fréquemment comme traduction erronée des mots anglais. 

Le modèle de traduction faisant suite est identique à ce qui a déjà été présenté et réalisé.  

In [13]:
# Création d'un nouveau dictionnaire basé sur le 1er dataset et ignorant certains mots français
# Le code qui suit reprend le code de la première partie qui a été utilisé pour créer le 1er dictionnaire
ban = ['le', 'la', 'les', 'une', 'est', 'en', "n'", "l'", 'de', 'ce']
liste_fr1_m = []
for i in range(len(liste_eng)):
    k = len(liste_fr[i].split()) - 1
    l = len(liste_eng[i].split()) - 1
    
    # Création de la liste des mots par phrase anglaise
    for j in range(len(liste_eng[i].split())):
        
        # Liste de mots pour le premier mot de la phrase anglaise
        if j == 0:
            liste_fr1_m.append([liste_fr[i].split()[0], liste_fr[i].split()[1], liste_fr[i].split()[2]])
            
        # Liste des mots pour le dernier mot de la phrase anglaise   
        elif j == l: 
            liste_fr1_m.append([liste_fr[i].split()[-3], liste_fr[i].split()[-2], liste_fr[i].split()[-1]])
            
        # Liste des mots pour les mots centraux de la phrase anglaise
        else:
            
            # Phrases anglaises et françaises avec le même nombre de mots
            if k == l:
                liste_fr1_m.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                
                
            # Phrases anglaises avec plus de mots que les françaises
            elif k < l:
                if 1 <= j < k:
                    liste_fr1_m.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                else:
                    liste_fr1_m.append([liste_fr[i].split()[-3], liste_fr[i].split()[-2], liste_fr[i].split()[-1]])
                
                
            # Phrases françaises avec plus de mots que phrases anglaises
            else:
                v = liste_fr[i].split()[j - 1]
                w = liste_fr[i].split()[j + 1]
                if (v and w not in ban) or (j == 1) or (j == l-1):
                    liste_fr1_m.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                elif v in ban and w not in ban:
                    liste_fr1_m.append([liste_fr[i].split()[j - 2], liste_fr[i].split()[j], liste_fr[i].split()[j + 1]])
                elif w in ban and v not in ban:
                    liste_fr1_m.append([liste_fr[i].split()[j - 1], liste_fr[i].split()[j], liste_fr[i].split()[j + 2]])
                elif v and w in ban:
                    liste_fr1_m.append([liste_fr[i].split()[j - 2], liste_fr[i].split()[j], liste_fr[i].split()[j + 2]])

print("Taille de la liste anglaise liste_eng1 :",len(liste_eng1))
print("Taille de la liste française liste_fr1_m :",len(liste_fr1_m))

Taille de la liste anglaise liste_eng1 : 1501574
Taille de la liste française liste_fr1_m : 1501574


In [14]:
arr_eng = np.array(liste_eng1).reshape(1501574,1)
arr_fr_m = np.array(liste_fr1_m)
arr_m = np.hstack((arr_eng, arr_fr_m))

arr1_m = arr_m[arr_m[:, 0].argsort()]


In [15]:
dico_trad_m = {}
dico_occ_nbr_m = {}
dico_occ_pctg_m = {}
k1 = 0 
for i in range(uniq_eng.shape[0]):
    k2 = uniq_eng[i, 1].astype('int')
    arr2_m = arr1_m[k1 : k1 + k2, 1:4]
    values, counts = np.unique(arr2_m, return_counts = True)
    dico_trad_m.update({uniq_eng[i, 0] : values[np.argmax(counts)]})
    dico_occ_nbr_m.update({uniq_eng[i, 0] : dict(zip(*np.unique(arr2_m, return_counts= True)))})
    dico_count_m = dict(zip(*np.unique(arr2_m, return_counts= True)))
    for key, value in dico_count_m.items():
        dico_count_m[key] = round(value / (k2 * 3) * 100, 2)
        dico_occ_pctg_m.update({uniq_eng[i, 0] : dico_count_m})
    k1 += k2

dico_occ_m = {}
for i in dico_trad_m.keys():
        dico_occ_m.update({i : list(dico_occ_nbr_m[i].keys())})

dico_trad_pctg_m = {}
for i in dico_trad_m.keys():
    for j in dico_occ_pctg_m[i].keys():
        if j == dico_trad_m[i]:
            dico_trad_pctg_m.update({i : [dico_trad_m[i], dico_occ_pctg_m[i][j]]})


dico_com1_m = {}
liste_nt_m = []
for i in dico_part.keys():
    if dico_trad_m[i] in dico_part[i]:
        dico_com1_m.update({i : [dico_trad_m[i], dico_trad_pctg_m[i][1]]})
    else:
        liste_nt_m.append(i)

# Analyse du pourcentage d'occurrence des mots choisis pour la traduction
liste_pctg_m = [dico_com1_m[i][1] for i in dico_com1_m.keys()]
print("Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 2eme dictionnaire ont une \
occurrence comprise entre", min(liste_pctg_m), "% et", max(liste_pctg_m), "%.\n")

# Analyse du score du 2eme dictionnaire
print("Les traductions proposées dans le 2eme dictionnaire correspondent à celles proposées par le dictionnaire de \
référence pour", len(dico_com1_m), "mots, soit dans", round(len(dico_com1_m)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots mal traduits, de", len(liste_nt_m), "mots :\n", liste_nt_m)


Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 2eme dictionnaire ont une occurrence comprise entre 14.27 % et 33.33 %.

Les traductions proposées dans le 2eme dictionnaire correspondent à celles proposées par le dictionnaire de référence pour 120 mots, soit dans 68.97 % des cas.

Liste des mots mal traduits, de 54 mots :
 ['animal', 'animals', 'automobile', 'bears', 'black', 'blue', 'california', 'did', 'dislike', 'driving', 'drove', 'during', 'eiffel', 'feared', 'field', 'football', 'fruit', 'going', 'grape', 'green', 'grocery', 'have', 'her', 'last', 'like', 'liked', 'limes', 'loved', 'might', 'most', 'new', 'next', 'nice', 'plan', 'plans', 'rusty', 'saw', 'school', 'shiny', 'spring', 'states', 'store', 'that', 'the', 'think', 'tower', 'translating', 'usually', 'want', 'wanted', 'went', 'where', 'white', 'would']


In [16]:
# Comparaison entre les différentes possibilités de traductions du 2e dictionnaire et celles du dictionnaire de référence
dico_lt_m = {}
for i in dico_part.keys():
    for j in range(len(dico_occ_m[i])):
        if dico_occ_m[i][j] in dico_part[i]:
            dico_lt_m.update({i : dico_occ_m[i]})
            break
            
liste_lnt_m = []
for i in dico_part.keys():
    if i not in dico_lt_m.keys():
        liste_lnt_m.append(i)

print("Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt_m), "mots, soit dans", 
      round(len(dico_lt_m)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt_m), "mots :\n", liste_lnt_m)


Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 160 mots, soit dans 91.95 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 14 mots :
 ['did', 'dislike', 'drove', 'feared', 'field', 'have', 'her', 'most', 'nice', 'plan', 'plans', 'saw', 'store', 'would']


En retirant les mots contenus dans la variable "ban", nous observons une augmentation du taux de bonnes réponses de 47% à 68%, ce qui valide cette piste d'amélioration. 

#### Analyse sur le seuil d'occurence

En dernier lieu, nous allons travailler sur la fixation d'un seuil d'occurence visant à mieux traduire les mots anglais.

1) méthode 1: conserver uniquement les traductions dont l'occurence est au-delà d'un certain seuil et vérifier leur bonne traduction par rapport à l'ensemble des mots en commun (174 mots).

2) méthode 2: conserver uniquement les traductions dont l'occurence est au-delà d'un certain seuil et vérifier leur bonne traduction par rapport à un nouveau dictionnaire de référence qui ne contient que les mots retenus (au-dessus du seuil précédemment mentionné).

Nous allons appliquer ces deux méthodes au dictionnaire obtenu. 

In [17]:
# Seuil d'occurrence pris en compte dans la création du dictionnaire de traduction commune

In [18]:
# Le seuil indiqué est en pourcentage
seuil = 25

In [19]:
# Dictionnaire 1
dico_com1_s = {}
for i in dico_com1.keys():
    if dico_com1[i][1]>=seuil:
        dico_com1_s.update({i : dico_com1[i][0]})

print("Pour un seuil de", seuil, "% d'occurrence minimum, les traductions proposées dans le 1er dictionnaire \
correspondent à celles de référence pour", len(dico_com1_s), "mots, soit dans", 
      round(len(dico_com1_s)/len(dico_part) * 100, 2), "% des cas.\n")

Pour un seuil de 25 % d'occurrence minimum, les traductions proposées dans le 1er dictionnaire correspondent à celles de référence pour 62 mots, soit dans 35.63 % des cas.



In [20]:
dico_lt_s = {}
for i in dico_part.keys():
    for j in range(len(dico_occ[i])):
        if dico_occ[i][j] in dico_part[i]:
            for k in dico_occ_pctg[i].keys():
                if dico_occ_pctg[i][k] >= seuil:
                    dico_lt_s.update({i : dico_occ_m[i]})
                    break
            
liste_lnt_s = []
for i in dico_part.keys():
    if i not in dico_lt_s.keys():
        liste_lnt_s.append(i)

print("Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt_s), "mots, soit dans", 
      round(len(dico_lt_s)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt_s), "mots :\n", liste_lnt_s)

Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 119 mots, soit dans 68.39 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 55 mots :
 ['and', 'animal', 'animals', 'are', 'august', 'beautiful', 'between', 'big', 'black', 'blue', 'busy', 'chinese', 'did', 'dislike', 'driving', 'drove', 'dry', 'during', 'english', 'feared', 'field', 'freezing', 'french', 'fruit', 'going', 'green', 'have', 'her', 'hot', 'little', 'may', 'most', 'next', 'nice', 'not', 'november', 'old', 'plan', 'plans', 'portuguese', 'quiet', 'red', 'rusty', 'saw', 'shiny', 'spanish', 'store', 'that', 'the', 'this', 'was', 'went', 'white', 'would', 'yellow']


In [21]:
# Dictionnaire 2
dico_com1_m_s = {}
for i in dico_com1_m.keys():
    if dico_com1_m[i][1]>=seuil:
        dico_com1_m_s.update({i : dico_com1_m[i][0]})

print("Pour un seuil de", seuil, "% d'occurrence minimum, les traductions proposées dans le 1er dictionnaire \
correspondent à celles de référence pour", len(dico_com1_m_s), "mots, soit dans", 
      round(len(dico_com1_m_s)/len(dico_part) * 100, 2), "% des cas.\n")

Pour un seuil de 25 % d'occurrence minimum, les traductions proposées dans le 1er dictionnaire correspondent à celles de référence pour 99 mots, soit dans 56.9 % des cas.



In [22]:
dico_lt_m_s = {}
for i in dico_part.keys():
    for j in range(len(dico_occ_m[i])):
        if dico_occ_m[i][j] in dico_part[i]:
            for k in dico_occ_pctg_m[i].keys():
                if dico_occ_pctg_m[i][k] >= seuil:
                    dico_lt_m_s.update({i : dico_occ_m[i]})
                    break
            
liste_lnt_m_s = []
for i in dico_part.keys():
    if i not in dico_lt_m_s.keys():
        liste_lnt_m_s.append(i)

print("Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt_m_s), "mots, soit dans", 
      round(len(dico_lt_m_s)/len(dico_part) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt_m_s), "mots :\n", liste_lnt_m_s)

Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 123 mots, soit dans 70.69 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 51 mots :
 ['and', 'animal', 'animals', 'are', 'august', 'beautiful', 'between', 'big', 'black', 'blue', 'chinese', 'did', 'dislike', 'driving', 'drove', 'dry', 'during', 'english', 'feared', 'field', 'freezing', 'french', 'fruit', 'going', 'green', 'have', 'her', 'little', 'most', 'next', 'nice', 'not', 'old', 'plan', 'plans', 'portuguese', 'quiet', 'red', 'rusty', 'saw', 'shiny', 'store', 'that', 'the', 'this', 'visit', 'was', 'went', 'white', 'would', 'yellow']


Nous constatons que le seuil baisse ce qui est normal car on compare avec l'ensemble des mots en commun mais le résultat est plus précis. 

In [23]:
# Seuil d'occurrence pris en compte dans la création du dictionnaire de référence

In [24]:
# Dictionnaire 1
dico_trad_s1 = {}
for i in dico_trad.keys():
    if dico_trad_pctg[i][1]>=seuil:
        dico_trad_s1.update({i : dico_trad[i]})
        
dico_part_s1 = {}
for i in dico_trad_s1.keys():
    if i in dico_part.keys():
        dico_part_s1.update({i : dico_cplt[i]})

print("Avec l'application d'un seuil minimum d'occurrence à", seuil, "%, on obtient un dictionnaire de",len(dico_trad_s1),
      "mots traduits.")
print("Ce dictionnaire a", len(dico_part_s1), "mots en commun avec le dictionnaire de référence.\n")

dico_com1_s1 = {}
liste_nt_s1 = []
for i in dico_part_s1.keys():
    if dico_trad_s1[i] in dico_part_s1[i]:
        dico_com1_s1.update({i : [dico_trad_s1[i], dico_trad_pctg[i][1]]})
    else:
        liste_nt_s1.append(i)

# Analyse du pourcentage d'occurrence des mots choisis pour la traduction
liste_pctg_s1 = [dico_com1_s1[i][1] for i in dico_com1_s1.keys()]
print("Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 1er dictionnaire ont une \
occurrence comprise entre", min(liste_pctg_s1), "% et", max(liste_pctg_s1), "%.\n")

# Analyse du score du 1er dictionnaire
print("Les traductions proposées dans le 1er dictionnaire correspondent à celles proposées par le dictionnaire de \
référence pour", len(dico_com1_s1), "mots, soit dans", round(len(dico_com1_s1)/len(dico_part_s1) * 100, 2),"% des cas.\n")
print("Liste des mots mal traduits, de", len(liste_nt_s1), "mots :\n", liste_nt_s1)


Avec l'application d'un seuil minimum d'occurrence à 25 %, on obtient un dictionnaire de 145 mots traduits.
Ce dictionnaire a 130 mots en commun avec le dictionnaire de référence.

Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 1er dictionnaire ont une occurrence comprise entre 25.38 % et 33.33 %.

Les traductions proposées dans le 1er dictionnaire correspondent à celles proposées par le dictionnaire de référence pour 62 mots, soit dans 47.69 % des cas.

Liste des mots mal traduits, de 68 mots :
 ['apple', 'apples', 'april', 'automobile', 'banana', 'bananas', 'bears', 'california', 'december', 'dislike', 'drove', 'eiffel', 'fall', 'feared', 'february', 'field', 'football', 'grape', 'grapefruit', 'grapes', 'grocery', 'have', 'her', 'january', 'july', 'june', 'last', 'lemon', 'lemons', 'like', 'liked', 'lime', 'limes', 'loved', 'mango', 'mangoes', 'march', 'might', 'new', 'nice', 'october', 'orange', 'oranges', 'peach', 'peaches', 'pear', 'pears', 'plans'

In [25]:
dico_lt_s1 = {}
for i in dico_part_s1.keys():
    for j in range(len(dico_occ[i])):
        if dico_occ[i][j] in dico_part_s1[i]:
            for k in dico_occ_pctg[i].keys():
                if dico_occ_pctg[i][k] >= seuil:
                    dico_lt_s1.update({i : dico_occ[i]})
                    break
            
liste_lnt_s1 = []
for i in dico_part_s1.keys():
    if i not in dico_lt_s1.keys():
        liste_lnt_s1.append(i)

print("Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt_s1), "mots, soit dans", 
      round(len(dico_lt_s1)/len(dico_part_s1) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt_s1), "mots :\n", liste_lnt_s1)

Les possibles traductions proposées dans le 1er dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 119 mots, soit dans 91.54 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 11 mots :
 ['dislike', 'drove', 'feared', 'field', 'have', 'her', 'nice', 'plans', 'saw', 'store', 'would']


In [26]:
# Dictionnaire 2
dico_trad_s2 = {}
for i in dico_trad_m.keys():
    if dico_trad_pctg_m[i][1]>=seuil:
        dico_trad_s2.update({i : dico_trad_m[i]})
        
dico_part_s2 = {}
for i in dico_trad_s2.keys():
    if i in dico_part.keys():
        dico_part_s2.update({i : dico_cplt[i]})

print("Avec l'application d'un seuil minimum d'occurrence à", seuil, "%, on obtient un dictionnaire de",len(dico_trad_s2),
      "mots traduits.")
print("Ce dictionnaire a", len(dico_part_s2), "mots en commun avec le dictionnaire de référence.\n")

dico_com1_s2 = {}
liste_nt_s2 = []
for i in dico_part_s2.keys():
    if dico_trad_s2[i] in dico_part_s2[i]:
        dico_com1_s2.update({i : [dico_trad_s2[i], dico_trad_pctg_m[i][1]]})
    else:
        liste_nt_s2.append(i)
        
# Analyse du pourcentage d'occurrence des mots choisis pour la traduction
liste_pctg_s2 = [dico_com1_s2[i][1] for i in dico_com1_s2.keys()]
print("Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 1er dictionnaire ont une \
occurrence comprise entre", min(liste_pctg_s2), "% et", max(liste_pctg_s2), "%.\n")

# Analyse du score du 2eme dictionnaire
print("Les traductions proposées dans le 2eme dictionnaire correspondent à celles proposées par le dictionnaire de \
référence pour", len(dico_com1_s2), "mots, soit dans", round(len(dico_com1_s2)/len(dico_part_s2) * 100, 2),"% des cas.\n")
print("Liste des mots mal traduits, de", len(liste_nt_s2), "mots :\n", liste_nt_s2)

Avec l'application d'un seuil minimum d'occurrence à 25 %, on obtient un dictionnaire de 148 mots traduits.
Ce dictionnaire a 134 mots en commun avec le dictionnaire de référence.

Les mots les plus fréquents choisis pour la traduction des mots anglais dans le 1er dictionnaire ont une occurrence comprise entre 25.2 % et 33.33 %.

Les traductions proposées dans le 2eme dictionnaire correspondent à celles proposées par le dictionnaire de référence pour 99 mots, soit dans 73.88 % des cas.

Liste des mots mal traduits, de 35 mots :
 ['automobile', 'bears', 'california', 'dislike', 'drove', 'eiffel', 'feared', 'field', 'football', 'grape', 'grocery', 'have', 'her', 'last', 'like', 'liked', 'limes', 'loved', 'might', 'new', 'nice', 'plans', 'saw', 'school', 'spring', 'states', 'store', 'think', 'tower', 'translating', 'usually', 'want', 'wanted', 'where', 'would']


In [27]:
dico_lt_m_s2 = {}
for i in dico_part_s2.keys():
    for j in range(len(dico_occ_m[i])):
        if dico_occ_m[i][j] in dico_part_s2[i]:
            for k in dico_occ_pctg_m[i].keys():
                if dico_occ_pctg_m[i][k] >= seuil:
                    dico_lt_m_s2.update({i : dico_occ_m[i]})
                    break
            
liste_lnt_m_s2 = []
for i in dico_part_s2.keys():
    if i not in dico_lt_m_s2.keys():
        liste_lnt_m_s2.append(i)

print("Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions \
proposées dans le dictionnaire de référence pour", len(dico_lt_m_s2), "mots, soit dans", 
      round(len(dico_lt_m_s2)/len(dico_part_s2) * 100, 2), "% des cas.\n")
print("Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de", 
      len(liste_lnt_m_s2), "mots :\n", liste_lnt_m_s2)

Les possibles traductions proposées dans le 2eme dictionnaire trouvent une correspondance dans les traductions proposées dans le dictionnaire de référence pour 123 mots, soit dans 91.79 % des cas.

Liste des mots pour lesquels aucune occurrence ne correspond à une traduction, de 11 mots :
 ['dislike', 'drove', 'feared', 'field', 'have', 'her', 'nice', 'plans', 'saw', 'store', 'would']


Le score de la deuxième méthode est plus élevé, ce qui est logique car le dictionnaire de référence nouvellement établi est plus bas. 

Conclusions générales: 

Le taux de réussite est très moyen en dépit des pistes d'amélioration étudiées (pré-étude sur l'influence de la taille de la fenêtre, retrait de certains mots en français, travail sur le seuil d'occurence). 

Cette méthode est peu valeureuse en dépit du faible nombre de mots uniques, du grand nombre d'occurences de ceux_ci et du fait que les phrases françaises étaient parfois un peu bancales (structuellement plus proches de l'anglais que du francais), par exemple : "votre moins aimé fruit est le raisin". 

En conclusion, cette méthode est trop fastieuse, peu fiable et ne peut pas être utilisé à plus grande échelle et poursuivue. 

Nous rejetons H0. 

